In [554]:
import pandas as pd
#import numpy as np
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from IPython.core.display import display, HTML
display(HTML('<style>.container { width:80% !important; }</style>'))
pd.set_option('display.max_columns', 100)


# Problem definition
1. Data mining and analysis to identify the number of Monthly Active Users
2. define user retention and determine what the characteristics(events)

## Read data from sources

In [391]:
url_users = 'https://s3-ap-southeast-2.amazonaws.com/propeller-static/data_challenge/users.csv'
url_events = 'https://s3-ap-southeast-2.amazonaws.com/propeller-static/data_challenge/events.csv'

users = pd.read_csv(url_users,index_col=0)
events = pd.read_csv(url_events)

# save to csv to not download 3gb again
events.to_csv('event.csv', index=False)
events = pd.read_csv('event.csv')

## Remove irrelevant fields, data clean up and transformation

In [392]:
events.describe(include='all')

,user_id,original_timestamp,context_page_search,event,event_text
count,4.866631e+06,4866631,4824088,4866631,4866631
unique,NaN,4865100,38033,86,86
top,NaN,2018-12-04 19:40:03.537,?project=prad7bb3e1&dataSet=ds3c344196,set_annotations_visibility,Set Annotations Visibility
freq,NaN,3,112618,790328,790328
mean,1.417033e+04,NaN,NaN,NaN,NaN
std,5.247930e+03,NaN,NaN,NaN,NaN
min,1.000000e+00,NaN,NaN,NaN,NaN
25%,1.124500e+04,NaN,NaN,NaN,NaN
50%,1.570600e+04,NaN,NaN,NaN,NaN
75%,1.811400e+04,NaN,NaN,NaN,NaN


In [393]:
# all other fields considered irrelevant for the task based on their uniqueness and context. 
events = events[[
    'user_id',
    'original_timestamp',
    'event',
]]

# dropping events with NA in user_id 
events = events.dropna(subset=['user_id'])

In [394]:
all_merged = events.merge(users, how='left', left_on='user_id', right_on=users.index)

In [ ]:
all_merged_time_index = all_merged.set_index(pd.DatetimeIndex(all_merged['original_timestamp'])).drop(['original_timestamp','context_page_search','event_text'], axis=1)

In [396]:
all_merged_time_index

,user_id,event,date_joined
original_timestamp,,,
2018-10-24 05:28:05.565,15689.0,timeline_selection,2018-07-02 12:52:40.606304+00
2018-10-24 05:28:14.603,14554.0,created_annotation,2018-05-02 02:31:12.513304+00
2018-10-24 05:28:14.695,14554.0,select_only,2018-05-02 02:31:12.513304+00
2018-10-24 05:28:55.394,14554.0,select_only,2018-05-02 02:31:12.513304+00
2018-10-24 05:29:26.516,14554.0,set_annotations_visibility,2018-05-02 02:31:12.513304+00
...,...,...,...
2019-04-24 04:54:15.719,20829.0,toggle_layer_visibility,2019-04-01 06:09:10.948459+00
2019-04-24 04:54:32.390,20829.0,toggle_layer_visibility,2019-04-01 06:09:10.948459+00
2019-04-24 04:55:09.734,19749.0,toggled_pane_layer,2019-02-18 00:49:11.325823+00


## Figure out Monthly Active Users
Users with at least one event during a month - considered as active

In [397]:
grouped_df_users = all_merged_time_index.groupby(pd.Grouper(freq='M'))['user_id'].nunique()
grouped_df_users

original_timestamp
2018-10-31     797
2018-11-30    1494
2018-12-31    1378
2019-01-31    1613
2019-02-28    1658
2019-03-31    1985
2019-04-30    1895
Freq: M, Name: user_id, dtype: int64

In [407]:
fig_users = px.bar(
    grouped_df_users, 
    x=grouped_df_users.index, 
    y=grouped_df_users.values,
    title="Monthly Active Users",
    labels = {
        'x':'Month',
        'y': 'User Quantity'
    }
)

fig_users.show()

## calculate events per month
I went with assumption that user actions frequency would give an indication of user's engagement. Hence, the most repeated events would indicate what users like to do, what functionality is in demand and brings them back to using the app. 

In [408]:
grouped_df_events = all_merged_time_index.groupby([pd.Grouper(freq='M'),'event'])['event'].count().unstack()

In [412]:
grouped_df_events

event,annotation_template_selected,archive_design_file_layer,change_annotation_template,chart_add_data_set,chart_remove_data_set,close_material_properties_modal,compass_interact_orbit,compass_interact_rotate,copied_measurement_to_current_dataset,copied_to_all_datasets,copied_to_dataset,copy_annotation_share_link,created_annotation,delete_dataset,delete_site,deleted_annotation,download_csv,download_design_file_source,download_elevation_heights_csv,download_selected_annotation_shape,download_source_photo,edited_annotation_template_saved,entered_into_edit_shape_state,exit_edit_shape_state,finish_onboarding_tour,fly_to_design_file_layer,full_screen_toggle,go_to_annotation,home_button_map_view,load_from_share,map_interact_camera,map_open_photo_pane,measurement_list_filtered,measurement_list_search,modify_design_file_layer_setting,new_annotation_template_saved,open_material_properties_modal,records_a_performance_measurement,redirected_from_app_dot,rename_design_file_layer,request_access,restore_design_file_layers,select_layer,select_only,selected_site,selected_visualiser_tool,set_annotation_category_is_collapsed,set_annotation_charted_item_id,set_annotation_visibility,set_annotation_visible_item_id,set_annotations_visibility,set_measurement_material,set_pane_visibility,set_primary_data_set,set_progress_to_design_chart_data,show_modal,skip_onboarding_tour,snapped_to_ground_checkbox_was_selected,snapped_to_ground_checkbox_was_unselected,thumbnail_selected,timeline_selection,toggle_annotation_item_calculator,toggle_annotation_item_contour,toggle_annotation_item_heat_map,toggle_contour_layers_contour,toggle_contour_layers_heat_map,toggle_layer_clamp_to_ground,toggle_layer_visibility,toggled_pane_layer,track_file_upload,track_pdf_download,update_annotation,update_contour_settings,update_dataset_name,update_layer_data_objer,update_site_name,update_user_setting,update_view_setting,visualiser_session_ended,visualiser_session_started,visualiser_time_to_interactive,volume_type_changed,zoom_in_button_in_map_view,zoom_in_button_in_photo_view,zoom_out_button_map_view,zoom_out_button_photo_view
original_timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-10-31,3390.0,NaN,325.0,499.0,159.0,105.0,1153.0,998.0,NaN,NaN,NaN,11.0,2854.0,5.0,34.0,1564.0,60.0,NaN,21.0,252.0,63.0,3.0,NaN,NaN,64.0,NaN,356.0,1959.0,1023.0,39.0,18381.0,995.0,NaN,NaN,NaN,10.0,106.0,118.0,20.0,NaN,17.0,NaN,2051.0,34676.0,1452.0,3447.0,1259.0,5.0,7538.0,5163.0,35779.0,194.0,15823.0,6767.0,NaN,1004.0,114.0,NaN,NaN,266.0,5233.0,40.0,30.0,68.0,387.0,656.0,82.0,NaN,5601.0,112.0,193.0,5229.0,1598.0,64.0,15.0,7.0,6094.0,268.0,2550.0,4927.0,2171.0,19.0,1755.0,186.0,1166.0,133.0
2018-11-30,12278.0,10.0,1372.0,1826.0,450.0,429.0,3526.0,2961.0,2.0,1.0,13.0,59.0,10454.0,84.0,108.0,5933.0,217.0,33.0,12.0,702.0,208.0,12.0,834.0,539.0,231.0,4.0,901.0,8680.0,3218.0,146.0,77949.0,3334.0,NaN,NaN,837.0,37.0,430.0,15.0,53.0,2.0,72.0,2.0,1930.0,107501.0,5907.0,12279.0,3480.0,NaN,26336.0,18410.0,112093.0,777.0,54095.0,25063.0,173.0,4614.0,405.0,NaN,NaN,519.0,29317.0,NaN,NaN,NaN,1377.0,2825.0,370.0,2085.0,21890.0,541.0,762.0,13397.0,5695.0,63.0,101.0,115.0,27102.0,1007.0,9379.0,18098.0,8454.0,NaN,5551.0,369.0,3327.0,195.0
2018-12-31,12412.0,8.0,1399.0,1572.0,428.0,339.0,3304.0,2955.0,227.0,11.0,311.0,37.0,10954.0,52.0,96.0,4902.0,195.0,57.0,15.0,745.0,541.0,2.0,4905.0,3309.0,79.0,29.0,811.0,8034.0,3778.0,135.0,66562.0,3205.0,90.0,95.0,1123.0,28.0,340.0,NaN,56.0,4.0,47.0,NaN,NaN,110472.0,5183.0,12421.0,3295.0,NaN,25685.0,17678.0,114711.0,544.0,50063.0,23293.0,758.0,3756.0,144.0,NaN,NaN,816.0,25013.0,NaN,NaN,NaN,1213.0,2754.0,209.0,14337.0,21007.0,498.0,848.0,13245.0,5145.0,66.0,51.0,76.0,27227.0,860.0,8498.0,16602.0,7901.0,NaN,5818.0,191.0,3545.0,132.0
2019-01-31,13923.0,49.0,1678.0,1637.0,433.0,478.0,3876.0,3427.0,183.0,14.0,478.0,51.0,12356.0,74.0,78.0,6072.0,264.0,107.0,28.0,1614.0,1451.0,13.0,6367.0,4431.0,229.0,17.0,986.0,10351.0,4088.0,156.0,7880

In [536]:
final = grouped_df_events.merge(grouped_df_users, left_on=grouped_df_events.index, right_on=grouped_df_users.index, how='left')

In [537]:
final.rename(columns={
    'key_0':'Month',
    'user_id':'Active Users Qty'
}, inplace=True)


In [538]:
final.set_index('Month', inplace=True)

# correlation calculation between the number of active users and the number of certain actions performed
Resulted chart produces 9 actions which have the highest correlation with user activity - hence an indication of events leading to user retention score increase. 

In [539]:
corr_results = final.corr(method ='pearson')
highly_correlated = corr_results.loc['Active Users Qty']>0.97
highly_coorrelated_mask = highly_correlated.where(highly_correlated==True).dropna().index.to_list()

In [540]:
final = final[highly_coorrelated_mask]

In [545]:
# I had to look up the code used for subplots generation in a loop and had no time left to change it, hence those 2 misc columns in order for it to work 
final['misc1']=0
final['misc2']=0

In [544]:
df=final

plot_rows=4
plot_cols=3
fig = make_subplots(rows=plot_rows, cols=plot_cols)

# add traces
x = 0
for i in range(1, plot_rows + 1):
    for j in range(1, plot_cols + 1):
        #print(str(i)+ ', ' + str(j))
        fig.add_trace(go.Scatter(x=df.index, y=df[df.columns[x]].values,
                                 name = df.columns[x],
                                 mode = 'lines'),
                     row=i,
                     col=j)

        x=x+1

# Format and show fig
fig.update_layout(height=1200, width=1200)
fig.show()

## Conclusion
In order to increase retention the following actions should be considered:
1. Promote zoom in/out functionality, make sure it works as intended and users are aware of it. 
2. Promote the importance of making annotations and make sure users are aware of annotations visibility settings
3. Promote functionality related to counours, i.e. setting and layers
4. Visualiser is the one the most important features of the app so as design file layer settings

The functionality highlighted above should have priority in bug fuxing, traning and promo materials.


#### P.S.
only after significant time invested I realised that retention cohort analysis would produce more accurate results =/ 
however I hope that I tried 'to work it out', plus implementation in code would do for the given task. 
if not, Arnold, I'm sorry and thanks for having a good chat with me and your professional approach, you are the champ. 